In [15]:
import tensorflow as tf
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
BASE_DATA_DIR = "../Data/Leaves"      
CSV_PATH = os.path.join(BASE_DATA_DIR, "all.csv")

In [3]:

GENERATIONS_DIR = "../Results/mobilenet"
os.makedirs(GENERATIONS_DIR, exist_ok=True)

In [4]:
df = pd.read_csv(CSV_PATH)
df = df.rename(columns={"id": "idx", "y": "label"})

In [5]:
df["image_path"] = df["idx"].apply(lambda x: os.path.join(BASE_DATA_DIR, x))

In [6]:
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42, stratify=df["label"])
train_df, val_df = train_test_split(train_val_df, test_size=0.1765, random_state=42, stratify=train_val_df["label"])

In [7]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_df["image_path"], train_df["label"])
).map(load_image).shuffle(500).batch(BATCH_SIZE)

val_ds = tf.data.Dataset.from_tensor_slices(
    (val_df["image_path"], val_df["label"])
).map(load_image).batch(BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices(
    (test_df["image_path"], test_df["label"])
).map(load_image).batch(BATCH_SIZE)


In [9]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(df["label"].nunique(), activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [10]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10


42/42 [==============================] - 18s 350ms/step - loss: 1.3565 - accuracy: 0.6814 - val_loss: 0.2728 - val_accuracy: 0.9720
Epoch 2/10
42/42 [==============================] - 14s 313ms/step - loss: 0.1163 - accuracy: 0.9895 - val_loss: 0.1149 - val_accuracy: 0.9860
Epoch 3/10
42/42 [==============================] - 14s 301ms/step - loss: 0.0493 - accuracy: 0.9970 - val_loss: 0.0706 - val_accuracy: 0.9895
Epoch 4/10
42/42 [==============================] - 13s 289ms/step - loss: 0.0269 - accuracy: 0.9993 - val_loss: 0.0543 - val_accuracy: 0.9930
Epoch 5/10
42/42 [==============================] - 13s 279ms/step - loss: 0.0176 - accuracy: 0.9993 - val_loss: 0.0467 - val_accuracy: 0.9930
Epoch 6/10
42/42 [==============================] - 13s 277ms/step - loss: 0.0126 - accuracy: 1.0000 - val_loss: 0.0433 - val_accuracy: 0.9930
Epoch 7/10
42/42 [==============================] - 13s 280ms/step - loss: 0.0094 - accuracy: 1.0000 - val_loss: 0.0360 - val_accuracy: 0.99

In [11]:
model_path = os.path.join(GENERATIONS_DIR, "mobilenet_flavia_model.h5")
model.save(model_path)
print(f"Model saved at {model_path}")


c:\Users\Mai\DeepFinal\tfenv_new\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved at ../Results/mobilenet\mobilenet_flavia_model.h5


In [12]:
y_true = np.concatenate([y for x, y in test_ds], axis=0)
y_pred_prob = model.predict(test_ds)
y_pred = np.argmax(y_pred_prob, axis=1)

report = classification_report(y_true, y_pred)
print(report)
with open(os.path.join(GENERATIONS_DIR, "classification_report.txt"), "w") as f:
    f.write(report)


9/9 [==============================] - 5s 410ms/step
              precision    recall  f1-score   support

           0       0.90      1.00      0.95         9
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10
           3       1.00      1.00      1.00        11
           4       1.00      1.00      1.00        11
           5       1.00      1.00      1.00         8
           6       1.00      1.00      1.00         9
           7       1.00      1.00      1.00         8
           8       1.00      1.00      1.00         8
           9       1.00      1.00      1.00         9
          10       0.89      1.00      0.94         8
          11       1.00      1.00      1.00         9
          12       1.00      1.00      1.00         8
          13       0.91      1.00      0.95        10
          14       1.00      0.89      0.94         9
          15       1.00      0.88      0.93         8
          16       1.00     

In [13]:
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=True, fmt="d", cmap="coolwarm")
plt.title("Confusion Matrix (Test Set)")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.savefig(os.path.join(GENERATIONS_DIR, "confusion_matrix.png"))
plt.close()

In [ ]:
plt.figure(figsize=(12,10))
num_classes = df["label"].nunique()
for i in range(num_classes):
    fpr, tpr, _ = roc_curve((y_true==i).astype(int), y_pred_prob[:,i])
    plt.plot(fpr, tpr, label=f"Class {i} (AUC={auc(fpr,tpr):.2f})")

plt.plot([0,1], [0,1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves (Test Set)")
plt.legend()
plt.savefig(os.path.join(GENERATIONS_DIR, "roc_auc.png"))
plt.close()

print("Evaluation metrics and plots saved in 'Results/movbilenet' folder")

Evaluation metrics and plots saved in 'generations/' folder
